In [13]:
from collections import deque
import heapq

def norm(state):
    return tuple(round(x, 4) for x in state)

def successors(state, capacities):
    C1, C2, C3 = capacities
    mins = [0.2*C1, 0.2*C2, 0.2*C3]  # safety thresholds
    succs = []
    S = list(state)
    for i in range(3):
        for j in range(3):
            if i == j:
                continue
            src = S[i]
            dst = S[j]
            src_min = mins[i]
            dst_cap = capacities[j]
            can_give = src - src_min
            can_accept = dst_cap - dst
            transfer = min(can_give, can_accept)
            if transfer > 1e-9:
                newS = S.copy()
                newS[i] -= transfer
                newS[j] += transfer
                action = (i, j, transfer)  # (src, dst, amount)
                succs.append((action, norm(newS)))
    return succs

def is_goal(state, target):
    return all(abs(state[i] - target[i]) < 1e-3 for i in range(3))

def bfs(start, target, capacities):
    start, target = norm(start), norm(target)
    q = deque()
    q.append((start, []))
    visited = set([start])
    while q:
        state, path = q.popleft()
        if is_goal(state, target):
            return path
        for action, ns in successors(state, capacities):
            if ns not in visited:
                visited.add(ns)
                q.append((ns, path + [action]))
    return None

def dfs(start, target, capacities, max_depth=20):
    start, target = norm(start), norm(target)
    stack = [(start, [])]
    visited = set([start])
    while stack:
        state, path = stack.pop()
        if is_goal(state, target):
            return path
        if len(path) >= max_depth:
            continue
        for action, ns in successors(state, capacities):
            if ns not in visited:
                visited.add(ns)
                stack.append((ns, path + [action]))
    return None

def heuristic(state, target):
    # Simple heuristic: total absolute difference
    return sum(abs(state[i] - target[i]) for i in range(3))

def astar(start, target, capacities):
    start, target = norm(start), norm(target)
    pq = []
    heapq.heappush(pq, (heuristic(start, target), 0, start, []))
    visited = set()
    while pq:
        est_cost, g, state, path = heapq.heappop(pq)
        if is_goal(state, target):
            return path
        if state in visited:
            continue
        visited.add(state)
        for action, ns in successors(state, capacities):
            if ns not in visited:
                new_g = g + 1
                f = new_g + heuristic(ns, target)
                heapq.heappush(pq, (f, new_g, ns, path + [action]))
    return None

def print_solution(name, start, capacities, path, target):
    print(f"\n==== {name} with step calculations ====")
    if path is None:
        print("No solution found.")
        return

    state = list(start)
    for step, (i, j, transfer) in enumerate(path, 1):
        state[i] -= transfer
        state[j] += transfer
        print(f"Operation {step} — Open valve ({i+1}->{j+1})")
        print(f"  Transfer = {transfer:.2f}")
        print(f"  New state = ({state[0]:.2f}, {state[1]:.2f}, {state[2]:.2f})\n")

    print(f"Reached target = {tuple(round(x,2) for x in state)}")
    print(f"Number of valve operations = {len(path)}")


In [9]:
capacities = (8.0, 5.0, 3.0)
initial = (8.0, 0.0, 0.0)
target = (2.4, 5.0, 0.6)

path_bfs = bfs(initial, target, capacities)
print_solution("BFS", initial, capacities, path_bfs, target)



==== BFS with step calculations ====
Operation 1 — Open valve (1->2)
  Transfer = 5.00
  New state = (3.00, 5.00, 0.00)

Operation 2 — Open valve (1->3)
  Transfer = 1.40
  New state = (1.60, 5.00, 1.40)

Operation 3 — Open valve (3->1)
  Transfer = 0.80
  New state = (2.40, 5.00, 0.60)

Reached target = (2.4, 5.0, 0.6)
Number of valve operations = 3


In [10]:

path_dfs = dfs(initial, target, capacities, max_depth=20)
print_solution("DFS", initial, capacities, path_dfs, target)




==== DFS with step calculations ====
Operation 1 — Open valve (1->3)
  Transfer = 3.00
  New state = (5.00, 0.00, 3.00)

Operation 2 — Open valve (3->2)
  Transfer = 2.40
  New state = (5.00, 2.40, 0.60)

Operation 3 — Open valve (1->2)
  Transfer = 2.60
  New state = (2.40, 5.00, 0.60)

Reached target = (2.4, 5.0, 0.6)
Number of valve operations = 3


In [11]:
path_astar = astar(initial, target, capacities)
print_solution("A*", initial, capacities, path_astar, target)


==== A* with step calculations ====
Operation 1 — Open valve (1->2)
  Transfer = 5.00
  New state = (3.00, 5.00, 0.00)

Operation 2 — Open valve (1->3)
  Transfer = 1.40
  New state = (1.60, 5.00, 1.40)

Operation 3 — Open valve (3->1)
  Transfer = 0.80
  New state = (2.40, 5.00, 0.60)

Reached target = (2.4, 5.0, 0.6)
Number of valve operations = 3
